In [1]:
import os
os.chdir("../")

In [2]:
pwd

'/home/dephinate/ASU/TextSummarization'

Model Evaluation

Define Configuration Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_path: Path
    model_path: Path
    metric_file_name: Path
    

Add a method to configuration manager to read configurations from config.yaml

In [4]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories
from TextSummarizer.entity import DataIngestionConfig, DataValidationConfig, DataTransformationConfig, ModelTrainerConfig


class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            param_filepath=PARAMS_FILE_PATH) -> None:

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])

    # Method to read Data Ingestion Config

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

    # Method to read Data Validation Config
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.status_file,
            all_required_files=config.all_required_files,
            local_data_folder=config.local_data_folder
        )
        return data_validation_config

    # Method to read Data Transformation Configuration
    def get_data_transformation_config(self) -> DataIngestionConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

    # Method to read model trainer configuration

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_training_epochs=params.num_training_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps

        )
        return model_trainer_config
    
    # Method to real model evaluation configuration
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            tokenizer_path= config.tokenizer_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name

        )
        return model_evaluation_config


Create model evaltuation entity

* a function to create batches
* a function to claculate scores
* * iterates over the list of batches
* * * tokenize
* * * model.gonerate
* * * evaluate- pass generated o/p and target to get score  

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk, load_metric
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")
import pandas as pd

/home/dephinate/miniconda3/envs/texts/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-02-02 20:06:34,908,INFO,config,PyTorch version 2.1.2 available.]


In [6]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig) -> None:
        self.config = config
        
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range (0,len(list_of_elements),batch_size):
            yield list_of_elements[i:i+batch_size]
    
    def calculate_metric_on_test_data(self,
                                      metric,
                                      dataset,
                                      column_text,
                                      column_summary,
                                      batch_size,
                                      model,
                                      tokenizer,
                                      device = "cuda" if torch.cuda.is_available() else "cpu"
                                      ):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text],batch_size))
        target_batches  = list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size))

        # For each batch in the list of article_batches

        for article_batche, target_batche in tqdm(
            zip(article_batches,target_batches,
                total = len(article_batches))

        ):
            
            inputs =  tokenizer(article_batche, max_length=1024, truncation= True, padding="max_length", return_tensors="pt")
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_mask=inputs["attention_mask"].to(device),
                                       length_penalty=0.8,
                                       num_beams=8,
                                       max_length=128)
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens= True, clean_up_tokenization_spaces=True) for s in summaries]

            decoded_summaries = [d.replace(""," ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references = target_batche)

        score=metric.compute()
        return score
    

    def evaluate(self,device = "cuda" if torch.cuda.is_available() else "cpu"):   
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        
        # load dataest
        dataset = load_from_disk(self.config.data_path)
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = load_metric("rouge")
        score = self.calculate_metric_on_test_data(metric=rouge_metric,
                                                   dataset=dataset,
                                                   column_text="article",
                                                   column_summary="highlights",
                                                   batch_size=2,
                                                   model=model_pegasus,
                                                   tokenizer=tokenizer,
                                                   device = "cuda" if torch.cuda.is_available() else "cpu"
                                                   )
        rouge_dict = dict(dict(rn,score[rn].mid.fmeasure) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index="pegasus")
        df.to_csv(self.config.metric_file_name, index=False)



In [9]:
try :
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluator = ModelEvaluation(config=model_evaluation_config)
    model_evaluator.evaluate()
except Exception as e:
    raise e


[2024-02-02 20:09:41,428,INFO,common,yaml file config/config.yaml loaded successfully]
[2024-02-02 20:09:41,430,INFO,common,yaml file params.yaml loaded successfully]
[2024-02-02 20:09:41,431,INFO,common,created directory at: artifacts]
[2024-02-02 20:09:41,440,INFO,common,created directory at: artifacts/model_evaluation]


OSError: Incorrect path_or_model_id: 'artifacts/model_trainer/tokenizer'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
for i in range(0,10,2):
    print(f"{i} {i+2}")

0 2
2 4
4 6
6 8
8 10
